In [125]:
import numpy as np

In [138]:
import nipy

In [126]:
pima = np.genfromtxt('pima-indians-diabetes.data', delimiter=',')

In [127]:
pima

array([[   6.   ,  148.   ,   72.   , ...,    0.627,   50.   ,    1.   ],
       [   1.   ,   85.   ,   66.   , ...,    0.351,   31.   ,    0.   ],
       [   8.   ,  183.   ,   64.   , ...,    0.672,   32.   ,    1.   ],
       ..., 
       [   5.   ,  121.   ,   72.   , ...,    0.245,   30.   ,    0.   ],
       [   1.   ,  126.   ,   60.   , ...,    0.349,   47.   ,    1.   ],
       [   1.   ,   93.   ,   70.   , ...,    0.315,   23.   ,    0.   ]])

In [128]:
def sghmc(Y, X, stogradU, M, eps, m, theta, C, V):
    n = X.shape[0]
    p = X.shape[1]
    
    # Randomly sample momentum
    r = np.random.multivariate_normal(np.zeros(M.shape[0]),M)[:,np.newaxis]
    
    # Precompute
    B = 0.5 * V * eps
    D = 2*(C-B)*eps
    Minv = np.linalg.inv(M)
    
    # Hamiltonian dynamics
    for i in range(m):
        theta = theta + (eps*np.linalg.inv(M) @ r).ravel()
        r = r - eps*stogradU(theta, Y, X, nbatch) - eps*C @ Minv @ r \
            + np.random.multivariate_normal(np.zeros(M.shape[0]),D).reshape(p, 1)

    return(theta)

In [129]:
nsample = 1000
nbatch = 10
M = np.identity(8)
C = 3 * np.identity(8)
eps = 0.1
m = 50
V = 4 * np.identity(8)

X = pima[:,0:8]
Y = pima[:,8]

In [130]:
def stogradU(theta, Y, X, nbatch):
    '''A function that returns the stochastic gradient. Adapted from Eq. 5.
    Inputs are:
        theta, the parameters
        Y, the response
        X, the covariates
        nbatch, the number of samples to take from the full data
    '''
    n = X.shape[0]
    batch_id = np.random.randint(0, n, size=nbatch)
    return -n/nbatch * X[batch_id,:].T @ (Y[batch_id][:,np.newaxis] - \
        1/(1+np.exp(-X[batch_id,:] @ theta[:,np.newaxis])))

In [131]:
# w noise, no mh
samples = np.zeros((nsample, 8))
theta = np.zeros(8)

for i in range(nsample):
    theta = sghmc(Y, X, stogradU, M, eps, m, theta, C, V)
    samples[i] = theta

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: overflow encountered in exp


In [140]:
nipy.information(theta)

AttributeError: module 'nipy' has no attribute 'information'

In [132]:
np.mean(samples, axis=0)

array([  5.48959678e+03,   5.29836461e+02,  -1.20909936e+03,
         8.82398149e-01,   8.19409432e+01,  -1.73818986e+02,
         4.29749670e+03,  -6.35323765e+02])